# Download the PubChem IDs, Filter Them
Get the SMILES strings from PubChem and then filter down based on certain criteria

In [1]:
from examol.simulate.initialize import write_xyz_from_mol
from examol.store.models import MoleculeRecord, Conformer
from tempfile import TemporaryDirectory
from multiprocessing.pool import Pool
from more_itertools import batched, peekable
from functools import partial
from shutil import copyfileobj
from rdkit.Chem import Descriptors
from rdkit import RDLogger, Chem
from pathlib import Path
from tqdm import tqdm
import requests
import shutil
import gzip
import yaml
import os
import re

Configuration

In [2]:
batch_size = 10000
max_molwt = 300
criteria_path = 'criteria/criteria-v3.1.yml'

Surpress complaints from RDKit

In [3]:
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

## Screening Function
Something to remove undesired molecules

In [4]:
def screen_molecules(
    to_screen: list[tuple[Chem.Mol, str]],
    max_molecular_weight: float,
    forbidden_smarts: list[str],
    required_smarts: list[str],
    allowed_elements: list[str],
    min_conjugation: int,
    allow_disconnected: bool,
) -> list[str]:
    """Screen molecules that pass molecular weights and substructure filters

    Args:
        to_screen: List of pairs of Molecules and XYZ strings to screen
        max_molecular_weight: Maximum molecular weight (g/mol)
        forbidden_smarts: List of SMARTS that cannot appear in a molecule
        required_smarts: List of SMARTS that must appear in the molecule
        allowed_elements: List of allowed elements
        allow_disconnected: Whether to allow non-bonded connections
    Returns: 
        List MoleculeRecords
    """
    # Pre-parse the SMARTS strings
    forbidden_smarts = [Chem.MolFromSmarts(s) for s in forbidden_smarts]
    required_smarts = [Chem.MolFromSmarts(s) for s in required_smarts]

    passed = []
    
    # Function for counting conjugation
    def count_conj(mol):
        """Count the number of conjugated bonds in a molecule

        Assumes they are all part of the same group

        Args:
            mol: Molecule to evaluate
        Returns:
            Number of conjugated bonds
        """

        # Check if any are conjugated
        is_conj = [bond.GetIsConjugated() for bond in mol.GetBonds()]

        # If any are conjugated, count the number of multiple bonds
        if any(is_conj):
            kekul_mol = Chem.Kekulize(mol, True)
            return sum(i and bond.GetBondTypeAsDouble() >= 2 for i, bond in zip(is_conj, mol.GetBonds()))
        else:
            return 0

    for mol, xyz in to_screen:
        # Skip if molecular weight is above a threshold
        mol_wt = Descriptors.MolWt(mol)
        if mol_wt > max_molecular_weight:
            continue

        # Skip if it contains a disallowed elements
        if any(atom.GetSymbol() not in allowed_elements for atom in mol.GetAtoms()):
                continue
        
        # Skip if it contains a disallowed group
        try:
            if any(mol.HasSubstructMatch(s) for s in forbidden_smarts):
                continue
        except:
            continue
        
        # Skip if it does not contain all of the allowed groups
        try:
            if not all(mol.HasSubstructMatch(s) for s in required_smarts):
                continue
        except:
            continue
            
        # Skip if does not have enough conjugated bonds
        n_conj = count_conj(mol)
        if n_conj < min_conjugation:
            continue
            
        # Check first if it has a non-bond
        smiles = Chem.MolToSmiles(mol)
        if '.' in smiles and not allow_disconnected:
            continue

        # Skip if molecule does not parse
        if mol is None:
            continue
            
        # Assemble the MoleculeRecord
        record = MoleculeRecord.from_identifier(smiles)
        conf = Conformer.from_xyz(xyz, source='pubchem')
        record.conformers.append(conf)
        passed.append(record.json())

    return passed

Load our criteria

In [5]:
with open(criteria_path) as fp:
    criteria = yaml.safe_load(fp)
criteria

{'allowed_elements': ['C', 'H', 'O', 'N', 'F', 'S', 'P', 'Cl', 'Br'],
 'forbidden_smarts': ['[CX3](=O)[OX1H0-,OX2H1]',
  '[CX3](=O)[OX2H1]',
  '[#6][#6][OX2H]',
  '[CX3](=[OX1])N=[CX3](=[OX1])',
  'O[CX4][F,Cl,Br,I]'],
 'required_smarts': ['a'],
 'min_conjugation': 3,
 'allow_disconnected': False}

Pin them to the function

In [6]:
screen_fun = partial(screen_molecules, max_molecular_weight=max_molwt, **criteria)

## Make functions to iterate from PubChem Data Files
PubChem supplies their database in SDF format, which includes both the SMILES string and a 3D geometry.

The Data Files are hosted on an [FTP server](https://ftp.ncbi.nlm.nih.gov/pubchem/Compound/CURRENT-Full/SDF/), which we can access via HTTP requests.

In [7]:
result = requests.get('https://ftp.ncbi.nlm.nih.gov/pubchem/Compound/CURRENT-Full/SDF/')

In [8]:
all_sdf_files = re.findall(r'>(Compound_[\d_]+\.sdf\.gz)</a>', result.content.decode())
print(f'Found {len(all_sdf_files)} SDF files')

Found 338 SDF files


In [9]:
def get_smiles_and_xyz(filename) -> str:
    """Iterate over all of the molecules in PubChem
    
    Args:
        filename: List of SDF fils to iterate through
    Yields:
        Pairs of RDKit mol, XYZ string
    """
    with TemporaryDirectory(prefix='smiles') as tmp:
        # Download and uncompress the DFT file
        file_path = Path(tmp) / 'molecule.sdf'
        sdf_url = f'https://ftp.ncbi.nlm.nih.gov/pubchem/Compound/CURRENT-Full/SDF/{filename}'
        with requests.get(sdf_url, stream=True) as req, file_path.open('wb') as fo:
            decompressed_fo = gzip.GzipFile(fileobj=req.raw)
            copyfileobj(decompressed_fo, fo)

        # Read all of the molecules, return the SMILES and XYZ as strings
        suppl = Chem.MultithreadedSDMolSupplier(str(file_path))
        for mol in suppl:
            if mol is not None:
                try:
                    yield mol, write_xyz_from_mol(mol)
                except (RuntimeError, ValueError):
                    continue
smiles_iter = peekable(get_smiles_and_xyz(all_sdf_files[0]))

## Filter PubChem
Do one block of molecules at a time, so that we can check point, then merge into a single file

In [10]:
output_dir = Path('output') / f'pubchem-{Path(criteria_path).name[:-4]}-molwt={max_molwt}'
output_dir.mkdir(exist_ok=True)
print(f'Writing to: {output_dir}')

Writing to: output/pubchem-criteria-v3.1-molwt=300


In [11]:
for i, filename in enumerate(tqdm(all_sdf_files, desc='files')):
    output_path = output_dir / f'batch-{i}.json.gz'
    if output_path.is_file(): 
        continue
    with gzip.open('tmp.json.gz', 'wt') as fp:
        smiles_iter = get_smiles_and_xyz(filename)
        for record_batch in map(screen_fun, batched(smiles_iter, batch_size)):
            for record in record_batch:
                print(record, file=fp)
    shutil.move('tmp.json.gz', output_path)

files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 338/338 [8:38:56<00:00, 92.12s/it]


Merge them all into a single file

In [12]:
out_path = output_dir.parent / f'{output_dir.name}.json.gz'
print(f'Saving collected data to {out_path}')

Saving collected data to output/pubchem-criteria-v3.1-molwt=300.json.gz


In [14]:
with gzip.open(out_path, 'wt') as fo:
    for path in tqdm(output_dir.glob("*.json.gz")):
        with gzip.open(path, 'rt') as fi:
            copyfileobj(fi, fo)

338it [22:47,  4.05s/it]
